### Stage 1 - Data extraction, labelling and sampling (PD dataset)

This notebook prepares a manageable modelling dataset from the raw `loan.csv` file:

- select a compact set of features
- create a default indicator (proxy) from `loan_status`
- filter to post-2011 vintages
- downsample non-defaults and create `sample_weight` to correct for sampling in later stages

In [ ]:
import pandas as pd
import numpy as np

#Configuration
RAW_PATH = "data/loan.csv"
OUT_PATH = "data/loans_stage1_sampled.csv"

MIN_ISSUE_YEAR = 2011
NON_DEFAULT_FRAC = 0.30
RANDOM_STATE = 42

DEFAULT_STATUSES = {
    "Charged Off",
    "Default",
    "Does not meet the credit policy. Status:Charged Off",
}

PD_FEATURES = [
    "loan_amnt", "term", "int_rate", "installment",
    "grade", "sub_grade", "purpose",
    "annual_inc", "emp_length", "home_ownership",
    "verification_status",
    "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "revol_util",
    "total_acc", "acc_now_delinq",
    "issue_d", "addr_state", "loan_status",
]

In [ ]:
#Load raw data (large file); select columns immediately to reduce memory usage
loan = pd.read_csv(RAW_PATH, low_memory=False)
loan = loan[PD_FEATURES].copy()

#Parse issue date and drop rows where it is not usable
loan["issue_d"] = pd.to_datetime(loan["issue_d"], format="%b-%Y", errors="coerce")
loan = loan.dropna(subset=["issue_d"]).copy()

print("Rows after column selection + issue_d parse:", len(loan))
print("Issue date range:", loan["issue_d"].min().date(), "->", loan["issue_d"].max().date())

Rows after column selection + issue_d parse: 2260668
Issue date range: 2007-06-01 -> 2018-12-01


In [ ]:
#Default indicator (label proxy)
loan["default_12m"] = loan["loan_status"].isin(DEFAULT_STATUSES).astype(int)
loan = loan.drop(columns=["loan_status"])

print("Default rate (proxy) on full selected sample:", float(loan["default_12m"].mean()))

Default rate (proxy) on full selected sample: 0.1160926770317446


In [ ]:
#Filter to vintages from MIN_ISSUE_YEAR onward
loan = loan[loan["issue_d"].dt.year >= MIN_ISSUE_YEAR].copy()

print("Rows after year filter:", len(loan))
print("Issue date range:", loan["issue_d"].min().date(), "->", loan["issue_d"].max().date())
print("Default rate after year filter:", float(loan["default_12m"].mean()))

Rows after year filter: 2239854
Issue date range: 2011-01-01 -> 2018-12-01
Default rate after year filter: 0.11577227801454916


### Missing values

We only drop observations missing essential modelling fields (`issue_d`, `loan_amnt`, `term`, `int_rate`, `installment`)

Other missing values are kept and handled later using imputation inside the modelling pipeline.

In [ ]:
#Drop rows missing essential fields (keep other missingness for pipeline imputation later)
essential = ["issue_d", "loan_amnt", "term", "int_rate", "installment", "default_12m"]
n_before = len(loan)
dr_before = float(loan["default_12m"].mean())
loan = loan.dropna(subset=essential).copy()

print("Rows kept after essential dropna:", len(loan), "/", n_before)
print("Default rate before:", dr_before)
print("Default rate after :", float(loan["default_12m"].mean()))

Rows kept after essential dropna: 2239854 / 2239854
Default rate before: 0.11577227801454916
Default rate after : 0.11577227801454916


In [ ]:
#Downsample non-defaults to reduce size; keep all defaults
defaults = loan[loan["default_12m"] == 1].copy()
non_defaults = loan[loan["default_12m"] == 0].copy()

non_defaults_sampled = non_defaults.sample(frac=NON_DEFAULT_FRAC, random_state=RANDOM_STATE)
loan_s = pd.concat([defaults, non_defaults_sampled], axis=0).reset_index(drop=True)

#Sample weights to recover population-like totals in later stages
loan_s["sample_weight"] = 1.0
loan_s.loc[loan_s["default_12m"] == 0, "sample_weight"] = 1.0 / NON_DEFAULT_FRAC

#Sort chronologically
loan_s = loan_s.sort_values("issue_d").reset_index(drop=True)

print("Sampled rows:", len(loan_s))
print("Sampled default rate:", float(loan_s["default_12m"].mean()))
print("Weight check (mean weight):", float(loan_s["sample_weight"].mean()))

Sampled rows: 853475
Sampled default rate: 0.3038319810187762
Weight check (mean weight): 2.624392044289522


In [ ]:
#Export
loan_s.to_csv(OUT_PATH, index=False)

print("Saved:", OUT_PATH)
print("Columns:", list(loan_s.columns))
loan_s.head()

Saved: data/loans_stage1_sampled.csv
Columns: ['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'purpose', 'annual_inc', 'emp_length', 'home_ownership', 'verification_status', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'acc_now_delinq', 'issue_d', 'addr_state', 'default_12m', 'sample_weight']


,loan_amnt,term,int_rate,installment,grade,sub_grade,purpose,annual_inc,emp_length,home_ownership,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,acc_now_delinq,issue_d,addr_state,default_12m,sample_weight
0,4800,36 months,6.17,146.40,A,A3,other,49200.0,8 years,MORTGAGE,...,6.0,0.0,14980,43.7,18.0,0.0,2011-01-01,FL,0,3.333333
1,7800,36 months,10.36,253.01,B,B5,debt_consolidation,52000.0,6 years,MORTGAGE,...,10.0,0.0,15851,58.5,21.0,0.0,2011-01-01,NC,0,3.333333
2,8000,36 months,6.91,246.69,A,A5,wedding,18000.0,3 years,RENT,...,7.0,0.0,120,1.4,19.0,0.0,2011-01-01,CA,0,3.333333
3,10000,36 months,6.54,306.68,A,A4,credit_card,58000.0,10+ years,RENT,...,12.0,0.0,13997,51.5,25.0,0.0,2011-01-01,IL,0,3.333333
4,2000,36 months,10.36,64.88,B,B5,vacation,26000.0,9 years,OWN,...,6.0,0.0,1378,44.5,27.0,0.0,2011-01-01,CA,0,3.333333


### Explanation of Default, Exposure, Loss and EL

default is the status of a borrower who failed to meet their credit obligations in full and on time.

exposure is the amount of capital exposed at the time of default.

loss is the amount of capital that is lost after a default occurs and net of any recoveries.

expected loss is the the amount of capital you expect to lose before knowing who will default, averaging across all borrowers.